<a href="https://colab.research.google.com/github/arnavjain2710/IITISoC24-ML-36/blob/main/IITISOC_Demo_with_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pip dependencies ek baar install karke session restart karlena, uske baad model interrupr kare bhi toh restart matt karna session galti se bhi

In [1]:
!pip install ffmpeg-python
!pip install ultralytics
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install diffusers
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [1]:
from IPython.display import HTML, clear_output
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

%cd /content/Wav2Lip

#download the pretrained model
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
!pip install git+https://github.com/elliottzheng/batch-face.git@master

!pip install ffmpeg-python mediapipe==0.8.11

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

%cd /
from ghc.l_ghc_cf import l_ghc_cf
%cd content

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

clear_output()
print("All set and ready!")

All set and ready!


Saare functions aur model loading, u need not run it again and again

In [2]:
from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    mimeType : 'audio/webm;codecs=opus'
  };
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
};

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... please wait!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve => {
  recordButton.onclick = () => {
    toggleRecording()

    sleep(2000).then(() => {
      resolve(base64data.toString())
    });
  }
});
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('output.wav', format='wav')  # Save as output.wav
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, _ = process.communicate(input=binary)

  # Read the saved .wav file
  sr, audio = wav_read('output.wav')

  return audio, sr

#import pandas as pd
import librosa
import matplotlib.pyplot as plt
import os
#import numpy as np

def create_mels(audio , image_file):
  fig = plt.figure(figsize=[1, 1])
  ax = fig.add_subplot(111)
  ax.axes.set_axis_off()
  fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
  y, sr = librosa.load(audio, sr=16000)
  mels = librosa.feature.melspectrogram(y=y, sr=sr)
  log_mels = librosa.power_to_db(mels, ref=np.max)
  librosa.display.specshow(log_mels, sr=sr, cmap='viridis')
  fig.savefig(image_file, dpi=400, bbox_inches='tight', pad_inches=0)
  plt.close(fig)
def create_png(input_path , output_path):
        input_file = input_path
        output_file = os.path.join(output_path, os.path.basename(input_file).replace('.wav', '.png'))
        create_mels(input_file, output_file)

from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image

model_path = '/content/best.pt'

# Load the YOLO model
model_emo = YOLO(model_path)

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline



device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

from transformers import BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',  # Choose from 'fp4' or 'nf4'
    bnb_4bit_compute_dtype=torch.float16  # Can be 'float16', 'bfloat16', 'float32', etc.
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are an assistant chatbot who continues the conversation understanding the user's emotion."},
]

def add_message(role, content):
    messages.append({"role": role, "content": content})

from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer_modelemo = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model_modelemo = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

def get_emotion(text):
  input_ids = tokenizer_modelemo.encode(text + '</s>', return_tensors='pt')

  output = model_modelemo.generate(input_ids=input_ids,
               max_length=2)

  dec = [tokenizer_modelemo.decode(ids) for ids in output]
  label = dec[0]
  return label






WARNING ⚠️ /content/best.pt appears to require 'dill', which is not in Ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.0s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1731: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [3]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import Video, display
import shutil

video_pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
video_pipe.scheduler = DPMSolverMultistepScheduler.from_config(video_pipe.scheduler.config)
video_pipe.enable_model_cpu_offload()


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

DO NOT RESTART SESSION AFTER RUNNING TTS CELL!

In [4]:
!pip install TTS


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 64.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other re

In [5]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

/usr/local/lib/python3.10/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:34<00:00, 42.7MiB/s]
100%|██████████| 1.87G/1.87G [00:34<00:00, 53.6MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 9.82kiB/s]

100%|██████████| 361k/361k [00:00<00:00, 798kiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 55.3iB/s]
 99%|█████████▊| 7.65M/7.75M [00:00<00:00, 42.5MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:11<00:00, 42.5MiB/s]

Isko baar baar run kr skte

In [8]:
audio, sr = get_audio()
import scipy.io.wavfile as wav
from scipy.signal import resample
import numpy as np

sampling_rate, data = wav.read("/content/output.wav")

new_sampling_rate = 16000

num_samples = int(len(data) * new_sampling_rate / sampling_rate)

resampled_data = resample(data, num_samples)

wav.write("resampled_audio.wav", new_sampling_rate, np.asarray(resampled_data, dtype=np.int16))

create_png('/content/resampled_audio.wav' , '/content')
audio_image_path='/content/resampled_audio.png'
test_image_path = audio_image_path
results = model_emo.predict(source=test_image_path)

predicted_class = results[0].names[results[0].probs.top1]
confidence = results[0].probs.top1conf

#print(f'Predicted Class: {predicted_class}, Confidence: {confidence:.2f}')

# Display the test image, can be removed, no need
img = Image.open(test_image_path)
plt.imshow(img)
plt.axis('off')
#plt.show()
emotion=predicted_class

sample1="/content/resampled_audio.wav"

result1 = pipe(sample1)
#print(result1)
#print(result1["text"])
prompt="text : " + result1["text"] + "emotion of user : " + emotion

add_message( "user" , prompt)
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
#print(tokenizer.decode(response, skip_special_tokens=True))
add_message("model" , tokenizer.decode(response, skip_special_tokens=True))


model_emo=get_emotion(str(response) )

tts.tts_to_file(#text=tokenizer.decode(response, skip_special_tokens=True),
                text=tokenizer.decode(response, skip_special_tokens=True),
                file_path="output.wav",
                speaker_wav="/content/male_voice (1).wav",
                language="en")
import librosa
import IPython.display as ipd
file_path = '/content/output.wav'
speech, sr = librosa.load(file_path, sr=None)

# Display the audio player
ipd.Audio(speech, rate=sr)


image 1/1 /content/resampled_audio.png: 640x640 neutral 0.58, disgust 0.21, fear 0.11, sad 0.07, happy 0.03, 6.0ms
Speed: 21.0ms preprocess, 6.0ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 > Text splitted to sentences.
["Sorry to hear that you're not feeling well.", "Would you like to talk about what's going on and how you're feeling?", 'Sometimes sharing your thoughts and emotions with someone can help you feel a bit better.']
 > Processing time: 16.663403749465942
 > Real-time factor: 0.979578266102152


In [10]:
# Mount Google Drive if it's not already mounted
if not os.path.isdir("/content/drive/MyDrive"):
    drive.mount('/content/drive', force_remount=True)

NameError: name 'drive' is not defined

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
shutil.move( '/content/output.wav','/content/drive/MyDrive/test.wav')

'/content/drive/MyDrive/test.wav'

In [ ]:
prompt = "Batman face, front view, he looks " + str(model_emo)
duration_in_seconds = 10
fps = 25  # Frames per second
total_frames = duration_in_seconds * fps
result = video_pipe(prompt, num_inference_steps=total_frames)
video_frames = result.frames

default_video_path = export_to_video(video_frames[0], fps=fps)

output_path = "/content/drive/MyDrive/generatedvid.mp4"

shutil.move(default_video_path, output_path)

print(f"Video saved to: {output_path}")

display(Video(output_path, embed=True))

  0%|          | 0/250 [00:00<?, ?it/s]

In [28]:
import os
import shutil
from google.colab import drive
from google.colab import files
from IPython.display import HTML, clear_output
from base64 import b64encode
import moviepy.editor as mp

def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

def get_video_resolution(video_path):
    """Function to get the resolution of a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return (width, height)

def resize_video(video_path, new_resolution):
    """Function to resize a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    fourcc = int(video.get(cv2.CAP_PROP_FOURCC))
    fps = video.get(cv2.CAP_PROP_FPS)
    width, height = new_resolution
    output_path = os.path.splitext(video_path)[0] + '_720p.mp4'
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    while True:
        success, frame = video.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, new_resolution)
        writer.write(resized_frame)
    video.release()
    writer.release()



# Remove previous input video if it exists
if os.path.isfile('/content/sample_data/input_vid.mp4'):
    os.remove('/content/sample_data/input_vid.mp4')

# Directly setting the video path
PATH_TO_YOUR_VIDEO = '/content/drive/MyDrive/generatedvid.mp4'

# Check if the video file exists
if not os.path.isfile(PATH_TO_YOUR_VIDEO):
    print("ERROR: File not found!")
    raise SystemExit(0)

# Ensure the video duration is less than 60 seconds
video_duration = mp.VideoFileClip(PATH_TO_YOUR_VIDEO).duration
if video_duration > 60:
    print("WARNING: Video duration exceeds 60 seconds. Please upload a shorter video.")
    raise SystemExit(0)

# Check and resize the video if necessary
video_resolution = get_video_resolution(PATH_TO_YOUR_VIDEO)
print(f"Video resolution: {video_resolution}")
if video_resolution[0] >= 1920 or video_resolution[1] >= 1080:
    print("Resizing video to 720p...")
    os.system(f"ffmpeg -i {PATH_TO_YOUR_VIDEO} -vf scale=1280:720 /content/sample_data/input_vid.mp4")
    PATH_TO_YOUR_VIDEO = "/content/sample_data/input_vid.mp4"
    print("Video resized to 720p")
else:
    print("No resizing needed")

# Copy the video to the working directory if not already there
if os.path.isfile(PATH_TO_YOUR_VIDEO):
    # Check if the source and destination files are the same
    if PATH_TO_YOUR_VIDEO != "/content/sample_data/input_vid.mp4":
        shutil.copyfile(PATH_TO_YOUR_VIDEO, "/content/sample_data/input_vid.mp4")
        print("Video copied to destination.")

    print("Input Video")
    # Display the video from the destination path
    showVideo("/content/sample_data/input_vid.mp4")
import os
from IPython.display import Audio
from IPython.core.display import display
import librosa
import soundfile as sf

# Remove previous input audio if it exists
if os.path.isfile('/content/sample_data/input_audio.wav'):
    os.remove('/content/sample_data/input_audio.wav')

def displayAudio():
    display(Audio('/content/sample_data/input_audio.wav'))

# Directly setting the audio path
PATH_TO_YOUR_AUDIO = '/content/drive/MyDrive/test.wav'

# Check if the audio file exists
if not os.path.isfile(PATH_TO_YOUR_AUDIO):
    print("ERROR: File not found!")
    raise SystemExit(0)

# Load audio with specified sampling rate
audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

# Save audio with specified sampling rate
sf.write('/content/sample_data/input_audio.wav', audio, sr, format='wav')

# Display the audio
clear_output()
displayAudio()


In [29]:
import locale

# Try setting the locale within Python
try:
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
except locale.Error:
    print("Warning: Could not set locale to en_US.UTF-8. You might encounter issues with text encoding.")

In [30]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [31]:
%cd /content/Wav2Lip

output_file_path = '/content/Wav2Lip/results/result_voice.mp4'

if os.path.exists(output_file_path):
    os.remove(output_file_path)

pad_top =  0
pad_bottom =  10
pad_left =  0
pad_right =  0
rescaleFactor =  1
nosmooth = True

use_hd_model = False
checkpoint_path = 'checkpoints/wav2lip.pth' if not use_hd_model else 'checkpoints/wav2lip_gan.pth'


if nosmooth == False:
  !python inference.py --checkpoint_path $checkpoint_path --face "../sample_data/input_vid.mp4" --audio "../sample_data/input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
else:
  !python inference.py --checkpoint_path $checkpoint_path --face "../sample_data/input_vid.mp4" --audio "../sample_data/input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth

if os.path.exists(output_file_path):
    clear_output()
    print("Final Video Preview")
    print("Download this video from", output_file_path)
    showVideo(output_file_path)
else:
    print("Processing failed. Output video not found.")

/content/Wav2Lip
Using cuda for inference.
Load checkpoint from: checkpoints/wav2lip.pth
Models loaded
Reading video frames...
Number of frames available for inference: 16
(80, 1251)
Length of mel chunks: 388
  0% 0/4 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/content/Wav2Lip/inference.py", line 323, in <module>
    main()
  File "/content/Wav2Lip/inference.py", line 254, in main
    for i, (img_batch, mel_batch, frames, coords) in enumerate(tqdm(gen,
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/content/Wav2Lip/inference.py", line 104, in datagen
    face_det_results = face_detect(frames) # BGR2RGB for CNN face detection
  File "/content/Wav2Lip/inference.py", line 81, in face_detect
    raise ValueError('Face not detected! Ensure the video contains a face in all the frames.')
ValueError: Face not detected! Ensure the video contains a face in all the frames.
Processing failed. Output video not f

To start a fresh conversation to make sure model forgets context, run this

In [7]:
messages = [
    {"role": "system", "content": "You are an assistant chatbot who continues the conversation understanding the user's emotion."},
]